In [1]:
library(simex)
library(reticulate)

In [2]:
merged_df <- read.csv("combined_mimic_smoking_status_0417.csv")
# head(merged_df)

In [3]:
fml <- 'mort_28_day ~ echo + first_careunit + age + gender + weight + saps + sofa + elix_score + vent + \
            vaso + icu_adm_weekday + icu_adm_hour + icd_chf + icd_afib + icd_renal + icd_liver + icd_copd + \
            icd_cad + icd_stroke + icd_malignancy + vs_heart_rate_first + vs_map_first + vs_temp_first + \
            lab_hemoglobin_first + lab_platelet_first + lab_wbc_first + lab_ph_first + lab_chloride_first + \
            lab_sodium_first + lab_bun_first + lab_bicarbonate_first + lab_pco2_first + lab_creatinine_first + \
            lab_potassium_first + lab_po2_first + lab_lactate_first + sedative + vs_cvp_flag + \
            lab_creatinine_kinase_flag + lab_bnp_flag + lab_troponin_flag + SMOKING_STATUS'
fml

[1] "mort_28_day ~ echo + first_careunit + age + gender + weight + saps + sofa + elix_score + vent + \n            vaso + icu_adm_weekday + icu_adm_hour + icd_chf + icd_afib + icd_renal + icd_liver + icd_copd + \n            icd_cad + icd_stroke + icd_malignancy + vs_heart_rate_first + vs_map_first + vs_temp_first + \n            lab_hemoglobin_first + lab_platelet_first + lab_wbc_first + lab_ph_first + lab_chloride_first + \n            lab_sodium_first + lab_bun_first + lab_bicarbonate_first + lab_pco2_first + lab_creatinine_first + \n            lab_potassium_first + lab_po2_first + lab_lactate_first + sedative + vs_cvp_flag + \n            lab_creatinine_kinase_flag + lab_bnp_flag + lab_troponin_flag + SMOKING_STATUS"

In [4]:
merged_df$SMOKING_STATUS <- as.factor(merged_df$SMOKING_STATUS)

In [5]:
glm_model = glm(as.formula(fml), data = merged_df, family = binomial, na.action = na.exclude)

In [6]:
summary(glm_model)


Call:
glm(formula = as.formula(fml), family = binomial, data = merged_df, 
    na.action = na.exclude)

Coefficients:
                             Estimate Std. Error z value Pr(>|z|)    
(Intercept)                 3.5542731  7.4974785   0.474 0.635455    
echo                       -0.1629170  0.1284231  -1.269 0.204585    
first_careunitSICU         -0.1714718  0.1725299  -0.994 0.320288    
age                         0.0196017  0.0049069   3.995 6.48e-05 ***
genderM                     0.2627325  0.1292458   2.033 0.042071 *  
weight                     -0.0076758  0.0028308  -2.712 0.006697 ** 
saps                        0.0941357  0.0169318   5.560 2.70e-08 ***
sofa                        0.2292723  0.0260908   8.787  < 2e-16 ***
elix_score                  0.0016398  0.0382643   0.043 0.965819    
vent                        0.2919105  0.2076957   1.405 0.159881    
vaso                        0.0096973  0.1592693   0.061 0.951450    
icu_adm_weekdaymonday       0.2885723  0.

In [29]:
matrix_error <- matrix(c(8/11, 0, 2/11, 1/11, 4/11, 4/11, 3/11, 0, 
                         1/16, 0, 14/16, 1/16, 1/63, 0, 1/63, 61/63), nrow=4)
matrix_error <- build.mc.matrix(matrix_error)
dimnames(matrix_error) <- list(levels(merged_df$SMOKING_STATUS), 
                               levels(merged_df$SMOKING_STATUS))
matrix_error

,1,2,3,4
1,0.7272727,0.3508917,0.0625,0.015873
2,0.0000000,0.3545172,0.0000,0.000000
3,0.1818182,0.2634643,0.8750,0.015873
4,0.0909091,0.0311268,0.0625,0.968254


In [8]:
tte_smoking_mcsimex <- mcsimex(glm_model, 
                               SIMEXvariable = "SMOKING_STATUS",
                               mc.matrix=matrix_error, 
                               asymptotic = FALSE)

In [9]:
summary(tte_smoking_mcsimex)

Call:
mcsimex(model = glm_model, SIMEXvariable = "SMOKING_STATUS", 
    mc.matrix = matrix_error, asymptotic = FALSE)

Naive model: 
glm(formula = as.formula(fml), family = binomial, data = merged_df, 
    na.action = na.exclude)

Simex variable : SMOKING_STATUS 
Misclassification matrix: 
          1         2      3        4
1 0.7272727 0.3508917 0.0625 0.015873
2 0.0000000 0.3545172 0.0000 0.000000
3 0.1818182 0.2634643 0.8750 0.015873
4 0.0909091 0.0311268 0.0625 0.968254

Number of iterations:  100 

Residuals: 
     Min.   1st Qu.    Median      Mean   3rd Qu.      Max. 
-0.926769 -0.211175 -0.065221  0.002541  0.174268  0.984207 

Coefficients: 

Jackknife variance: 
                             Estimate Std. Error t value Pr(>|t|)    
(Intercept)                 3.544e+00  7.607e+00   0.466  0.64131    
echo                       -1.569e-01  1.299e-01  -1.207  0.22741    
first_careunitSICU         -1.561e-01  1.750e-01  -0.892  0.37242    
age                         2.026e-02

In [ ]:
# plot(tte_smoking_mcsimex)

### Calculating Risk Ratio using MC-SIMEX

In [10]:
merged_df_0 <- read.csv("combined_mimic_smoking_status_0417.csv")

In [11]:
data <- replace(merged_df_0["echo"], merged_df_0["echo"]>0, 0) 
# print(data)

In [12]:
merged_df_0["echo"] <- data
merged_df_0$SMOKING_STATUS <- as.factor(merged_df_0$SMOKING_STATUS)

In [13]:
predictions_0 = predict(tte_smoking_mcsimex, merged_df_0, type="response")

In [14]:
p0_total = sum(predictions_0, na.rm=T)
print(p0_total)

[1] 669.8559


In [15]:
merged_df_1 <- read.csv("combined_mimic_smoking_status_0417.csv")

In [16]:
data <- replace(merged_df_1["echo"], merged_df_1["echo"]>-1, 1)

In [17]:
merged_df_1["echo"] <- data
merged_df_1$SMOKING_STATUS <- as.factor(merged_df_1$SMOKING_STATUS)

In [18]:
predictions_1 = predict(tte_smoking_mcsimex, merged_df_1, type="response")

In [19]:
p1_total = sum(predictions_1, na.rm=T)
print(p1_total)

[1] 625.6658


In [20]:
rr = p1_total / p0_total
print(rr)

[1] 0.9340305


### Bootstrapping Risk Ratio Via Error Rate Matrices

In [21]:
require("reticulate")


In [25]:
rr_arr <- list()
for (x in 0:9){
    print(x)
    tmp_m_error <- matrix_script_reader(paste("INSERT FILE PATH",
                                              as.character(x),".pkl", 
                                              sep=''))
    tmp_m_error <- build.mc.matrix(tmp_m_error, method="log")
    dimnames(tmp_m_error) <- list(levels(merged_df$SMOKING_STATUS), levels(merged_df$SMOKING_STATUS))
    
    tryCatch({boot_mcsimex <- mcsimex(glm_model, 
                                      SIMEXvariable = "SMOKING_STATUS", 
                                      mc.matrix=tmp_m_error, 
                                      asymptotic = FALSE)}
            , error = function(e) {tmp_m_error <- build.mc.matrix(tmp_m_error, method="jlt"); 
                                   boot_mcsimex <- mcsimex(glm_model, 
                                                           SIMEXvariable = "SMOKING_STATUS", 
                                                           mc.matrix=tmp_m_error, 
                                                           asymptotic = FALSE)} )
    
    merged_df_0 <- read.csv("combined_mimic_smoking_status_0417.csv")
    data_0 <- replace(merged_df_0["echo"], merged_df_0["echo"]>0, 0) 
    merged_df_0["echo"] <- data_0
    merged_df_0$SMOKING_STATUS <- as.factor(merged_df_0$SMOKING_STATUS)
    predictions_0 = predict(boot_mcsimex, merged_df_0, type="response")
    p0_total = sum(predictions_0, na.rm=T)
    
    merged_df_1 <- read.csv("combined_mimic_smoking_status_0417.csv")
    data_1 <- replace(merged_df_1["echo"], merged_df_1["echo"]>-1, 1)
    merged_df_1["echo"] <- data_1
    merged_df_1$SMOKING_STATUS <- as.factor(merged_df_1$SMOKING_STATUS)
    predictions_1 = predict(boot_mcsimex, merged_df_1, type="response")
    p1_total = sum(predictions_1, na.rm=T)

    rr = p1_total / p0_total
    rr_arr <- append(rr_arr, rr)
    print(rr)
    
}

[1] 0
[1] 0.932313
[1] 1
[1] 0.9331153
[1] 2
[1] 0.9316244
[1] 3
[1] 0.9316425
[1] 4
[1] 0.9361585
[1] 5
[1] 0.931594
[1] 6
[1] 0.931594
[1] 7
[1] 0.9327472
[1] 8
[1] 0.9323461
[1] 9
[1] 0.9308459


In [26]:
quantile(unlist(rr_arr))

0%       25%       50%       75%      100% 
0.9308459 0.9316016 0.9319778 0.9326469 0.9361585

In [45]:
quantile(unlist(rr_arr), c(.025, 0.975))

2.5%     97.5% 
0.9310142 0.9354738

### Bootstrapping Risk Ratio via Sampling Dataframe

In [36]:
rr_df_arr <- list()
for (x in 0:9){

    sampled_df <- merged_df[sample(nrow(merged_df), size=nrow(merged_df), replace=TRUE), ]
    sampled_df$SMOKING_STATUS <- as.factor(sampled_df$SMOKING_STATUS)
    sampled_glm_model <- glm(as.formula(fml), 
                             data = sampled_df, 
                             family = binomial, 
                             na.action = na.exclude)
    
    matrix_error <- matrix(c(8/11, 0, 2/11, 1/11, 4/11, 4/11, 3/11, 
                             0, 1/16, 0, 14/16, 1/16, 1/63, 0, 1/63, 
                             61/63), nrow=4)
    matrix_error <- build.mc.matrix(matrix_error)
    dimnames(matrix_error) <- list(levels(merged_df$SMOKING_STATUS), levels(merged_df$SMOKING_STATUS))
    
    tryCatch({sampled_mc_simex_model <- mcsimex(sampled_glm_model, 
                                                SIMEXvariable = "SMOKING_STATUS", 
                                                mc.matrix=matrix_error, 
                                                asymptotic = FALSE)}
            , error = function(e) {sampled_df <- merged_df[sample(nrow(merged_df), 
                                                                  size=nrow(merged_df)-1, 
                                                                  replace=TRUE), ]; 
                                   sampled_df$SMOKING_STATUS <- as.factor(sampled_df$SMOKING_STATUS)
                                   sampled_glm_model <- glm(as.formula(fml), 
                                                            data = sampled_df, 
                                                            family = binomial, 
                                                            na.action = na.exclude); 
                                   sampled_mc_simex_model <- mcsimex(sampled_glm_model, 
                                                                     SIMEXvariable = "SMOKING_STATUS", 
                                                                     mc.matrix=matrix_error, 
                                                                     asymptotic = FALSE)} )
    
    sampled_df_0 <- sampled_df
    sampled_data_0 <- replace(sampled_df_0["echo"], sampled_df_0["echo"]>0, 0) 
    sampled_df_0["echo"] <- sampled_data_0
    sampled_df_0$SMOKING_STATUS <- as.factor(sampled_df_0$SMOKING_STATUS)
    sampled_predictions_0 = predict(sampled_mc_simex_model, sampled_df_0, type="response")
    sampled_p0_total = sum(sampled_predictions_0, na.rm=T)
    
    sampled_df_1 <- sampled_df
    sampled_data_1 <- replace(sampled_df_1["echo"], sampled_df_1["echo"]>-1, 1)
    sampled_df_1["echo"] <- sampled_data_1
    sampled_df_1$SMOKING_STATUS <- as.factor(sampled_df_1$SMOKING_STATUS)
    sampled_predictions_1 = predict(sampled_mc_simex_model, sampled_df_1, type="response")
    sampled_p1_total = sum(sampled_predictions_1, na.rm=T)

    sample_rr = sampled_p1_total / sampled_p0_total
    rr_df_arr <- append(rr_df_arr, sample_rr)
    print(sample_rr)
}

[1] 0.8662869
[1] 0.8672094
[1] 0.9519129
[1] 0.9516234
[1] 0.8619673
[1] 0.9478805
[1] 0.9554614
[1] 1.008167
[1] 1.059316
[1] 0.9511795


In [39]:
quantile(unlist(rr_df_arr))

0%       25%       50%       75%      100% 
0.8619673 0.8873772 0.9514015 0.9545742 1.0593156

In [46]:
quantile(unlist(rr_df_arr), c(.025, 0.975))

2.5%     97.5% 
0.8629392 1.0478072

### Bootstrapping Risk Ratio combining both strategies

In [43]:
rr_combined_arr <- list()
for (x in 0:9){

    sampled_df <- merged_df[sample(nrow(merged_df), size=nrow(merged_df), replace=TRUE), ]
    sampled_df$SMOKING_STATUS <- as.factor(sampled_df$SMOKING_STATUS)
    sampled_glm_model <- glm(as.formula(fml), data = sampled_df, family = binomial, na.action = na.exclude)
    
    
    for (y in 0:9){
        
        tmp_m_error <- matrix_script_reader(paste("INSERT FILE PATH",
                                                  as.character(y),".pkl", 
                                                  sep=''))
        tmp_m_error <- build.mc.matrix(tmp_m_error, method="log")
        dimnames(tmp_m_error) <- list(levels(sampled_df$SMOKING_STATUS), levels(sampled_df$SMOKING_STATUS))
        
        tryCatch({sampled_mc_simex_model <- mcsimex(sampled_glm_model, 
                                                    SIMEXvariable = "SMOKING_STATUS", 
                                                    mc.matrix=tmp_m_error, 
                                                    asymptotic = FALSE)}
            , error = function(e) {sampled_df <- merged_df[sample(nrow(merged_df), 
                                                                  size=nrow(merged_df)-1, 
                                                                  replace=TRUE), ]; 
                                   sampled_df$SMOKING_STATUS <- as.factor(sampled_df$SMOKING_STATUS)
                                   sampled_glm_model <- glm(as.formula(fml), 
                                                            data = sampled_df, 
                                                            family = binomial, 
                                                            na.action = na.exclude);
                                   tmp_m_error <- build.mc.matrix(tmp_m_error, method="jlt");
                                   sampled_mc_simex_model <- mcsimex(sampled_glm_model, 
                                                                     SIMEXvariable = "SMOKING_STATUS", 
                                                                     mc.matrix=tmp_m_error, 
                                                                     asymptotic = FALSE)} )
        sampled_df_0 <- sampled_df
        sampled_data_0 <- replace(sampled_df_0["echo"], sampled_df_0["echo"]>0, 0) 
        sampled_df_0["echo"] <- sampled_data_0
        sampled_df_0$SMOKING_STATUS <- as.factor(sampled_df_0$SMOKING_STATUS)
        sampled_predictions_0 = predict(sampled_mc_simex_model, sampled_df_0, type="response")
        sampled_p0_total = sum(sampled_predictions_0, na.rm=T)

        sampled_df_1 <- sampled_df
        sampled_data_1 <- replace(sampled_df_1["echo"], sampled_df_1["echo"]>-1, 1)
        sampled_df_1["echo"] <- sampled_data_1
        sampled_df_1$SMOKING_STATUS <- as.factor(sampled_df_1$SMOKING_STATUS)
        sampled_predictions_1 = predict(sampled_mc_simex_model, sampled_df_1, type="response")
        sampled_p1_total = sum(sampled_predictions_1, na.rm=T)

        sample_rr = sampled_p1_total / sampled_p0_total
        rr_combined_arr <- append(rr_combined_arr, sample_rr)
        print(sample_rr)
           
    }
    
}

[1] 0.8498759
[1] 0.8547966
[1] 0.8494987
[1] 0.8505417
[1] 0.857892
[1] 0.8506836
[1] 0.8506836
[1] 0.8522792
[1] 0.8505662
[1] 0.8507709
[1] 0.8829385
[1] 0.8815192
[1] 0.8804538
[1] 0.881803
[1] 0.8782177
[1] 0.8834176
[1] 0.8834176
[1] 0.8824677
[1] 0.8830197
[1] 0.8812688
[1] 0.9178056
[1] 0.9169759
[1] 0.9148318
[1] 0.9172119
[1] 0.9174504
[1] 0.9179122
[1] 0.9179122
[1] 0.9168176
[1] 0.9168748
[1] 0.9166274
[1] 0.8943567
[1] 0.8936341
[1] 0.8909847
[1] 0.8910933
[1] 0.8968332
[1] 0.8923147
[1] 0.8923147
[1] 0.8946779
[1] 0.8939735
[1] 0.8914655
[1] 0.9099752
[1] 0.9074819
[1] 0.9072579
[1] 0.9080405
[1] 0.9102136
[1] 0.9089579
[1] 0.9089579
[1] 0.9092853
[1] 0.9083634
[1] 0.909059
[1] 0.9191958
[1] 0.9199039
[1] 0.9164768
[1] 0.9179599
[1] 0.9192645
[1] 0.918949
[1] 0.918949
[1] 0.9189645
[1] 0.9166585
[1] 0.918716
[1] 1.009724
[1] 1.014891
[1] 1.010643
[1] 1.012132
[1] 1.017634
[1] 1.007861
[1] 1.007861
[1] 1.008069
[1] 1.00921
[1] 1.008941
[1] 0.9159847
[1] 0.9121385
[1] 0.911

In [44]:
quantile(unlist(rr_combined_arr))

0%       25%       50%       75%      100% 
0.8494987 0.8933042 0.9158779 0.9832850 1.0176342

In [47]:
quantile(unlist(rr_combined_arr), c(0.025, 0.975))

2.5%     97.5% 
0.8505533 1.0114251